In [1]:
from tensorflow.contrib.tensorboard.plugins import projector
import os
import tensorflow as tf
import numpy as np
import pandas as pd
import cv2 as cv

/home/sha_maayan/python-environments/env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
LOGDIR = os.path.join(os.getcwd(), "logs/models/")
LABELS = os.path.join(os.getcwd(), "labels.tsv")

PATH_TO_CROPPED_IMAGES = '../images/cropped/'
PATH_TO_LABELS = '../annotations/cropped_color_label.csv'
CLASSES = 6
H, W, CH = 36, 64, 3

In [3]:
df = pd.read_csv(PATH_TO_LABELS)
X = []
y = []
for _, row in df.iterrows():
    img = cv.imread(os.path.join(PATH_TO_CROPPED_IMAGES, row['filename']))
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    img = cv.resize(img, (W, H))
    X.append(img)
    y.append(row['label'])

X = np.asarray(X)
y = np.asarray(y)

print('Shape of X: {}'.format(X.shape))

Shape of X: (104, 36, 64, 3)


In [4]:
# Note: Must be very consistent in naming, checkpoint file has to be named the same thing as tensor (I think)
pred_probs = tf.Variable(X, name='pred_probs')
 
# metadata_path = os.path.join(log_dir, 'metadata.tsv')
# metadata_cols = ['title', 'class']
# with open(metadata_path, 'w') as met_fi:
#     met_fi.write('%s\n' % '\t'.join(metadata_cols))
#     for rn in range(act_res.shape[0]):
#         cur_col = [all_titles[rn], '%d' % act_res[rn]]
#         met_fi.write('%s\n' % '\t'.join(cur_col))
 
config = projector.ProjectorConfig()
embedding = config.embeddings.add()
embedding.tensor_name = pred_probs.name
embedding.metadata_path = LABELS

In [5]:
# # Use the same LOG_DIR where you stored your checkpoint.
# init_op = tf.variables_initializer([pred_probs])
# with tf.Session() as sess:
#     sess.run(init_op)
#     saver = tf.train.Saver({'pred_probs': pred_probs})
#     saver.save(sess, os.path.join(LOGDIR, "model.ckpt-0600"))
 
#     summary_writer = tf.summary.FileWriter(LOGDIR)
 
#     # The next line writes a projector_config.pbtxt in the LOG_DIR. TensorBoard will
#     # read this file during startup.
#     projector.visualize_embeddings(summary_writer, config)

In [6]:
features = tf.Variable(X, name='features')

with tf.Session() as sess:
    saver = tf.train.Saver([features])

    sess.run(features.initializer)
    saver.save(sess, os.path.join(LOGDIR, 'model.ckpt'))
    
    config = projector.ProjectorConfig()
    # One can add multiple embeddings.
    embedding = config.embeddings.add()
    embedding.tensor_name = features.name
    # Link this tensor to its metadata file (e.g. labels).
    embedding.metadata_path = LABELS
    # Comment out if you don't want sprites
#     embedding.sprite.image_path = os.path.join(LOGDIR, 'sprite_4_classes.png')
#     embedding.sprite.single_image_dim.extend([img_data.shape[1], img_data.shape[1]])
    # Saves a config file that TensorBoard will read during startup.
    projector.visualize_embeddings(tf.summary.FileWriter(LOGDIR), config)